## *Phrase Detection algorithm*

##The basic idea is to get word2vec skip gram model embedding using a big text data (here I have used a a gutenberg novel).
##Since the above embeddings capture both symantic and syntactic relation between word we can use the similarity between these vector to form a cluster with a different clustering approach.
##Once we get the embeddings if a new sentence is fed to the model the model converts the word to embeddings. Initially if there are n word the number of clusters are n.
##Then we can compare similarities between the clusters using euclidean distance, cosine similarity, etc calculated between newly obtained vectors using weights as shown below for the consecutive clusters in the sentence. This done to maintain a sequence. The two consecutive clusters having maximum similarity are merged together to form one cluster.
##This process is repeated until we get a single cluster. This would give us a tree like structure.
##Appropriate phrases can be selected from the above tree at appropriate level since the phrases are not unique.
##The approach is similar to agglomerative hierarchical clustering
## Weights for left cluster=[n.....,4,3,2,1]/(nx(n+1)/2)
## Weights for right cluster=[1,2,3,4,.....n]/(nx(n+1)/2)

Data Reading and cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization
from gensim.models import Word2Vec
from scipy import spatial
data=open('/content/drive/MyDrive/Datasets/66568-0.txt').read()                        #write the location of the text file here
for x in list('!"#$%&\'()*+,-/:;<=>?@[\]^_`{|}~'):
  data=data.replace(x,'')
data=data.replace('\n','.')
data=data.split('.')

Training of word2vec skip gram model using window size 5 and minimum count to be 1

In [ ]:
for i in range(len(data)):
  data[i]=data[i].split()
model=Word2Vec(data,size=100,min_count=1,sg=1)

Embedding a new sentence

In [ ]:
t='There was no one to tell her that Philibert had once wanted to marry Bianca'.split()  #Type the sentence manually here
#t=input().split()                                                                       #Or use this line of code for input
embedding=[]
for word in t:
  embedding.append(model.wv.word_vec(word))

Clustering the words to form phrases

In [ ]:
new_embedding=[[np.array(element)] for element in embedding]
new_cluster_word=[[element] for element in t.copy()]
clustering=[new_cluster_word.copy()]
def cluster_weight_left(vectors):
  n=len(vectors)
  weights=np.array([i*2/(n*(n+1)) for i in range(1,n+1)])
  final_vector=np.dot(np.array(vectors).T,weights)
  return final_vector
def cluster_weight_right(vectors):
  n=len(vectors)
  weights=np.array([(n-i+1)*2/(n*(n+1)) for i in range(1,n+1)])
  final_vector=np.dot(np.array(vectors).T,weights)
  return final_vector
j=1
similarity=[]
while len(new_embedding)>1:
  sim=[]
  for i in range(len(new_embedding)-1):
    #sim.append(np.linalg.norm(new_embedding[i][-1]-new_embedding[i+1][0]))                                                       #cluster using nearest inner two words using Euclidean Distance
    #sim.append(np.linalg.norm(sum(new_embedding[i])/len(new_embedding[i])-sum(new_embedding[i+1])/len(new_embedding[i+1])))      #cluster using average of vector in the two consecutive clusters using Euclidean Distance
    sim.append(np.linalg.norm(cluster_weight_left(new_embedding[i])-cluster_weight_right(new_embedding[i+1])))                    #cluster using weighted average of vector in the two consecutive clusters using Euclidean Distance
    #sim.append(spatial.distance.cosine(cluster_weight_left(new_embedding[i]),cluster_weight_right(new_embedding[i+1])))          #cluster using weighted average of vector in the two consecutive clusters using Euclidean Distance
    #sim.append(spatial.distance.cosine(sum(new_embedding[i])/len(new_embedding[i]),sum(new_embedding[i+1])/len(new_embedding[i+1]))) #cluster using average of vector in the two consecutive clusters using Euclidean Distance
  sim=np.array(sim)
  pos=np.argmax(sim)
  new_embedding[pos]+=new_embedding[pos+1]
  new_embedding.pop(pos+1)
  new_cluster_word[pos]=new_cluster_word[pos]+new_cluster_word[pos+1].copy()
  new_cluster_word.pop(pos+1)
  clustering.insert(0,new_cluster_word.copy())
  similarity.insert(0,sum(sim)/len(sim))
  j+=1
for line in clustering:
  for element in line[:-1]:
    print(' '.join(element),end='<-------->')
  print(' '.join(line[-1]),end='\n'*2)

There was no one to tell her that Philibert had once wanted to marry Bianca

There was no one to tell her that<-------->Philibert had once wanted to marry Bianca

There was no one to tell her that<-------->Philibert had once<-------->wanted to marry Bianca

There was no<-------->one to tell her that<-------->Philibert had once<-------->wanted to marry Bianca

There was no<-------->one to<-------->tell her that<-------->Philibert had once<-------->wanted to marry Bianca

There was no<-------->one to<-------->tell her that<-------->Philibert had<-------->once<-------->wanted to marry Bianca

There was no<-------->one to<-------->tell her that<-------->Philibert had<-------->once<-------->wanted to marry<-------->Bianca

There was no<-------->one to<-------->tell her<-------->that<-------->Philibert had<-------->once<-------->wanted to marry<-------->Bianca

There was<-------->no<-------->one to<-------->tell her<-------->that<-------->Philibert had<-------->once<-------->wanted to marry<

From the above tree line structure approriate level will give us the appropriate phrase

In [ ]:
"""
import matplotlib.pyplot as plt
plt.plot(similarity)
plt.show
""" 

'\nimport matplotlib.pyplot as plt\nplt.plot(similarity)\nplt.show\n'

In [ ]:
"""
sim_slope=np.array([similarity[i+1]-similarity[i] for i in range(len(similarity)-1)])
plt.plot(sim_slope)
plt.show
"""

'\nsim_slope=np.array([similarity[i+1]-similarity[i] for i in range(len(similarity)-1)])\nplt.plot(sim_slope)\nplt.show\n'

In [ ]:
"""
best_level=sim_slope[1:].argmin()-2
for element in clustering[best_level]:
  print(' '.join(element),end='<-------->')
"""

"\nbest_level=sim_slope[1:].argmin()-2\nfor element in clustering[best_level]:\n  print(' '.join(element),end='<-------->')\n"